In [1]:
import os
from collections import OrderedDict
import re

In [2]:
fh = open('Pollen5_SNPs_with_na.Het.chr1.test', 'wt')

with open('Pollen5_SNPs_with_na.Het.chr1.txt') as f:
    for line in f:
        line = line.rstrip()
        line = line.replace('NA','N')
        lst = line.split('\t')
        
        if lst[0] == '':
            fh.write('\t'.join(['#chr','position']+lst[2:])+'\n')
            continue
            
        
        chrom = lst[0].split(':')[0]
        position = lst[0].split(':')[1]
        
        fh.write('\t'.join([chrom, position]+lst[2:])+'\n')
        
fh.close()
        

In [38]:
hetsnp_filter_file = 'hmmPol'

pairwisehmm = './crossover_hmm/pairwisehmm -n 20'
hmm_para = 'crossover_hmm/pairwise_2states.hmm'

In [39]:
Data = []
Pos = []
with open(hetsnp_filter_file+'.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        
        if line.startswith('#'):
            Heads = lst[2:]
            continue
            
        Pos.append(lst[0:2])
        Data.append(lst[2:])

In [40]:
for ancestor in range(len(Heads)):
    ancestor_id = Heads[ancestor]
    crossover_file = hetsnp_filter_file+'.'+ancestor_id+'.crossover'
    hiddenseq_file = hetsnp_filter_file+'.'+ancestor_id+'.hiddenseq'
    CO = open(crossover_file, 'wt')
    HS = open(hiddenseq_file, 'wt')
    
    for offspring in range(len(Heads)):
        if offspring == ancestor:
            continue
        
        offspring_id = Heads[offspring]
        temp_file = hetsnp_filter_file+'.temp'
        OUT = open(temp_file, 'wt')
        
        OUT.write('#chr\tposition\t'+ancestor_id+'\t'+offspring_id+'\n')

        for pos in range(len(Pos)):
            chrom = Pos[pos][0]
            position = Pos[pos][1]
            ancestor_base = Data[pos][ancestor]
            offspring_base = Data[pos][offspring]
            
            if (ancestor_base == 'N') or (offspring_base == 'N'):
                continue
            OUT.write('\t'.join([chrom, position, ancestor_base, offspring_base])+'\n')
                
        OUT.close()
        
        os.system(pairwisehmm+' '+hmm_para+' '+temp_file)
        
        with open(temp_file+'.crossover') as f:
            lines1 = f.read().splitlines()
            
        with open(temp_file+'.hiddenseq') as f:
            lines2 = f.read().splitlines()
        
        #os.system('rm '+temp_file+' '+temp_file+'.hiddenseq '+temp_file+'.crossover')
        
        for line1 in lines1:
            line1 = line1.rstrip()
            
            if line1.startswith('#'):
                continue
                
            CO.write('\t'.join([ancestor_id, offspring_id, line1])+'\n')
            
        for line2 in lines2:
            line2 = line2.rstrip()
            
            if line2.startswith('ChrID'):
                continue
                
            HS.write('\t'.join([ancestor_id, offspring_id, line2])+'\n')
            
    CO.close()
    HS.close()
    
    

In [6]:
crossover_file

'Pollen5_SNPs_with_na.Het.chr1.test.pol45.crossover'

In [7]:
######################################################################################
################################# filter crossovers ##################################

#chr_len_file = 'hg19_for_oocyte.fa.len'
#human_gap_file = 'hg19.fa.gap.centromere'


pairwise_crossovers_file = crossover_file

ChrLen = OrderedDict()
CO = OrderedDict()
MidCent = OrderedDict()
TerCent = OrderedDict()

#read_chrlen(chr_len_file, %ChrLen)
#read_chrgap(human_gap_file, %MidCent, %TerCent)
pairwise_crossovers_file = 'Pollen5_SNPs_with_na.Het.chr1.test.pol43.crossover'



with open(pairwise_crossovers_file) as f:
    for line in f:
        line = line.rstrip()
        
        lst = line.split('\t')
        
        sample = lst[1]
        pos = int(lst[3])
        
        if sample not in CO.keys():
            CO[sample] = OrderedDict()
        CO[sample][pos] = lst
        
        
fh = open(pairwise_crossovers_file+'.filter', 'wt')
Min_dist_two_crossover = 1000000 ### default 1000000

for sample_id in sorted(CO.keys()):
    sample_p = CO[sample_id]
    #print (sorted(sample_p.keys()))
    
    while 1:
        Pos = sorted(sample_p.keys())
        if len(Pos) < 2:
            break
        has_delete = 0
    
        for i in range(1,len(Pos)):
            if Pos[i] - Pos[i-1] < Min_dist_two_crossover:
                del sample_p[Pos[i]]
                del sample_p[Pos[i-1]]
                has_delete = 1
                break
                
        if has_delete == 0:
            break
            
    
    Pos = sorted(sample_p.keys())
    
    if len(Pos) >= 1:
        if int(Pos[0]) < Min_dist_two_crossover:
            del sample_p[Pos[0]]
            
        #if Pos[-1] > ChrLen[ChrID]-Min_dist_two_crossover:
        #    del sample_p[Pos[-1]]
            
    for pos in sorted(sample_p.keys()):
        pos_p = sample_p[pos]
        pos_p[7] = '1'
        line = '\t'.join(pos_p)
        #print (line)
        fh.write(line+'\n')
        
fh.close()

In [8]:
#####################################################################################
################################# infer crossovers ##################################

pairwise_crossovers_file = 'Pollen5_SNPs_with_na.Het.chr1.test.pol45.crossover.filter'

Distance_cutoff = 1000000
Reference_crossover_ratio = 0.6
Sample_total = 5
CO = []

with open(pairwise_crossovers_file) as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        
        sample = lst[1]
        pos = lst[3]
        
        CO.append(lst)
        
CO.sort(key=lambda x: int(x[3]))


fh = open(pairwise_crossovers_file+'.cluster', 'wt')

if len(CO) >= 1:
    cluster = []
    Confident_crossover_id = 1
    cluster.append(CO[0])
    
    for i in range(1, len(CO)):
        this_pos = int(CO[i][3])
        cluster_end = int(cluster[-1][3])
        
        if this_pos - cluster_end < Distance_cutoff:
            cluster.append(CO[i])
            
        else:
            Count = OrderedDict()
            
            for j in range(len(cluster)):
                sample = cluster[j][1]
                Count[sample] = cluster[j]
                
            number = len(Count)
            pos_centered = 0
            
            for sample in Count.keys():
                pos_centered += int(Count[sample][3])
                
            pos_centered = int(pos_centered/number)
            ratio = number/Sample_total
            
            if ratio >= Reference_crossover_ratio:
                fh.write(' '.join(map(str,['\n**',Confident_crossover_id,'Confident Reference Crossover:',pos_centered,ratio]))+'\n')
                Confident_crossover_id += 1
                
            else:
                fh.write('\n## Reference crossover:  '+str(pos_centered)+' '+str(ratio)+'\n')
                
            for p in cluster:
                line = '\t'.join(p)
                fh.write(line+'\n')
                
            cluster = []
            cluster.append(CO[i])
                
    if (len(cluster)> 0):
        Count = OrderedDict()
            
        for j in range(len(cluster)):
            sample = cluster[j][1]
            Count[sample] = cluster[j]
                
        number = len(Count)
        pos_centered = 0
            
        for sample in Count.keys():
            pos_centered += int(Count[sample][3])
                
        pos_centered = int(pos_centered/number)
        ratio = number/Sample_total
            
        if ratio >= Reference_crossover_ratio:
            fh.write(' '.join(map(str,['\n**',Confident_crossover_id,'Confident Reference Crossover:',pos_centered,ratio]))+'\n')
            Confident_crossover_id += 1
                
        else:
            fh.write('\n## Reference crossover:  '+str(pos_centered)+' '+str(ratio)+'\n')
                
        for p in cluster:
            line = '\t'.join(p)
            fh.write(line+'\n')
            
fh.close()

In [9]:
#############################################################################
######################### merge pairwise phased snps ########################

from glob import glob
hetsnp_filter_file = 'Pollen5_SNPs_with_na.Het.chr1.test'

Crossover = OrderedDict()
SAMPLE = OrderedDict()

files = sorted(glob('*.crossover.filter.cluster'))

for file in files:
    mtch = re.search(hetsnp_filter_file+'.(\w+)'+'.crossover.filter.cluster', file)
    sample = mtch.group(1)
    SAMPLE[sample] = 1
    
    with open(file) as f:
        for line in f:
            line = line.rstrip()
            if line.startswith('**'):
                lst = line.split(' ')
                
                crossover_pos = int(lst[-2])
                ratio = float(lst[-1])
                
                if sample not in Crossover.keys():
                    Crossover[sample] = OrderedDict()
                
                Crossover[sample][crossover_pos] = ratio
                

In [10]:
##create %Phase data structure

ChrLen = OrderedDict()
chrID = 'chr1'
ChrLen[chrID] = 29341521

Phase = OrderedDict()

for sample in SAMPLE.keys():
    ary = []
    last_pos = 1
    last_status = 1
    
    if sample in Crossover.keys():
        sample_p = Crossover[sample]
        
        for cur_pos in sorted(sample_p.keys()):
            ary.append([last_pos, cur_pos, last_status])
            last_pos = cur_pos
            last_status = -last_status
            
    ary.append([last_pos, ChrLen[chrID], last_status])
    Phase[sample] = ary
    
    number = len(ary)
    
    print ('sample\t'+str(number)+'\n')  

sample	2

sample	3

sample	2

sample	4

sample	1



In [11]:
#### Ö»¶ÁÈ¡Ïž°ûBºÍCµÄÊýŸÝ£¬²»ÒªanenploidyµÄchromosomes, ÒòÎªÕâÊÇ×öphasingËùÐèÒªµÄ

Heads = OrderedDict()
HetSNP = OrderedDict()
SNP = OrderedDict()

hetsnp_filter_file = 'Pollen5_SNPs_with_na.Het.chr1.test'

with open(hetsnp_filter_file) as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        
        if line.startswith('#'):
            for i in range(4,len(lst)):
                Heads[i] = lst[i]
            continue
            
        chrom = lst[0]
        pos = int(lst[1])
        
        HetSNP[pos] = [lst[2], lst[3]]  ### ?
        
        for i in range(4, len(lst)):
            sample = Heads[i]
            
            if sample not in SNP.keys():
                SNP[sample] = OrderedDict()
            
            SNP[sample][pos] = lst[i]

In [12]:
##phasing each sample to be one haplotype

for sample in SNP.keys():
    sample_p = SNP[sample] ### will change SNP
    
    for pos in sorted(sample_p.keys()):
        base = sample_p[pos]
        
        if base != 'N':
            phase_p = Phase[sample]
            for p in phase_p:
                if pos >= int(p[0]) and pos <= int(p[1]):
                    phase_status = p[2]
                    break
                    
            if phase_status == -1:
                allele_p = HetSNP[pos]
                for allele in allele_p:
                    if allele != base:
                        base = allele
                        break
    
        sample_p[pos] = base
                
            

In [13]:
##find the reference sample that has the highest similarity to the other samples

ref_ratio = -1

for reference in sorted(SNP.keys()):
    print ('Reference: '+ reference)
    average_ratio = 0
    number = 0
    
    for query in sorted(SNP.keys()):
        if reference == query:
            continue
            
        sample_p = SNP[query]
        total_num = 0
        same_num = 0
        
        for pos in sorted(sample_p.keys()):
            base = sample_p[pos]
            ref_base = SNP[reference][pos]
            
            if base != 'N' and ref_base != 'N':
                total_num += 1
                
                if base == ref_base:
                    same_num += 1
                    
        ratio = same_num/total_num
            
        if ratio > 0.5:
            relationship = 1
        else:
            relationship = -1
                
        if relationship == -1:
            ratio = 1 - ratio
                
        average_ratio += ratio
        ratio = int(ratio * 100)
        number += 1
            
    average_ratio = average_ratio/number
        
    if ref_ratio == -1 or average_ratio > ref_ratio:
        ref_sample = reference
        ref_ratio = average_ratio
            
    print (average_ratio)
        
print ('\nReference sample: '+ref_sample+'\t'+str(ref_ratio))

        

Reference: pol41
0.9919951047337887
Reference: pol42
0.9822634437509241
Reference: pol43
0.9928235622474422
Reference: pol44
0.9888438903555724
Reference: pol45
0.9936205374418811

Reference sample: pol45	0.9936205374418811


In [14]:
##calculate the relationship between reference sample haptype and the other cells
##change all the cells' haploptype to the reference' haplotype

for sample in sorted(SNP.keys()):
    if sample == ref_sample:
        continue
        
    sample_p = SNP[sample]
    total_num = 0
    same_num = 0
        
    for pos in sorted(sample_p.keys()):
        base = sample_p[pos]
        ref_base = SNP[ref_sample][pos]
            
        if base != 'N' and ref_base != 'N':
            total_num += 1
                
            if base == ref_base:
                same_num += 1
                    
    ratio = same_num/total_num
            
    if ratio > 0.5:
        relationship = 1
    else:
        relationship = -1
                
    if relationship == -1:
        ratio = 1 - ratio
        
    if ratio < 0.85:
        print ('Warning: bad consistence between '+ref_sample+' and '+sample+': '+str(ratio)+'\t'+str(relationship))
    else:
        print ('Good consistence between '+ref_sample+' and '+sample+': '+str(ratio)+'\t'+str(relationship))
        
        
    if relationship == -1:
        for pos in sorted(sample_p.keys()):
            base = sample_p[pos]
            ref_base = SNP[ref_sample][pos]
            if base == 'N':
                continue
            allele_p = HetSNP[pos]
            
            for allele in allele_p:
                if allele != base:
                    alt_base = allele
                    break
                    
            sample_p[pos] = alt_base

Good consistence between pol45 and pol41: 0.9988693112220861	1
Good consistence between pol45 and pol42: 0.9871620307333203	1
Good consistence between pol45 and pol43: 0.9982195666606313	-1
Good consistence between pol45 and pol44: 0.9902312411514865	-1


In [15]:
fh = open('test.txt', 'wt')
for ke in SNP.keys():
    fh.write(ke+'\n')
    for k in SNP[ke].keys():
        fh.write(str(SNP[ke][k])+'\n')
        
fh.close()

In [ ]:
##merge all the haplotyp to form the donor's phased hetSNP sets
fh = open(hetsnp_filter_file+'.phased', 'wt')

fh.write("ChrID\tpos\thap1\thap2\tcover\tconsistence%\n")

p = SNP[ref_sample]

for pos in sorted(p.keys()):
    Count = OrderedDict()
    total = 0
    
    for sample in sorted(SNP.keys()):
        base = SNP[sample][pos]
        if base != 'N':
            if base not in Count.keys():
                Count[base] = 0
            Count[base] += 1
            total += 1
            
    if len(Count) == 0:
        print (str(pos)+' no genotype')
        continue
            
    ary = sorted(Count, key=Count.__getitem__, reverse=True)
    major_base = ary[0]
    
    if total < 2 or Count[major_base]/total < 0.6:
        print (str(pos)+' low confidence')
        continue
        
    major_rate = int(Count[major_base]/total*100)
        
    allele_p = HetSNP[pos]
    for allele in allele_p:
        if allele != major_base:
            alt_base = allele
            break
            
    fh.write('\t'.join(map(str,[chrID, pos, major_base, alt_base, total, major_rate]))+'%\n')
    
fh.close()

In [322]:
from operator import itemgetter

In [356]:
x = {'x':1,'a':4,'z':3}
#sorted_x = [ke for ke insorted(x.items(), key=itemgetter(1), reverse=True)

In [364]:
Count.keys()

odict_keys([])

In [363]:
sorted(Count.values(), reverse=True)

[]

In [357]:
sorted(x, key=x.__getitem__, reverse=True)

['a', 'z', 'x']

In [160]:
for ke, val in Crossover.items():
    for k in val.keys():
        print (ke+'\t'+k)

pol41	26530917
pol42	5778526
pol42	23291445
pol43	23460964
pol44	3193425
pol44	5383005
pol44	24995179
